### STEP 1
#### Generating .thy files from templates for different valid theory files

- Three cipher templates: Simon, Speck, PRESENT
- Templates are parameterized on (block_size, key_size) for uniformity
- Simon: (block_size, key_size) → derive n, m, num_rounds
- Speck: (block_size, key_size) → look up (α, β, num_rounds) via helper functions
- Present: (block_size, key_size) → fixed num_rounds = 31, fixed S-box, P-layer

#### SIMON AND SPECK  VARIANTS GENERATION
- 10 Simon variants generated given SIMON's and SPECK's valid setups
- Generated from simon_template.thy and speck_template.thy with (block_size, key_size) as parameters


In [1]:
import os

# Simon & Speck variants: (block_size, key_size)
simon_speck_setups = [
    (32, 64),   # block_size, key_size
    (48, 72),   
    (48, 96),
    (64, 96),
    (64, 128),
    (96, 96),
    (96, 144),
    (128, 128),
    (128, 192),
    (128, 256),
]

TEMPLATE_PATH = ["simon_template.thy", "speck_template.thy"]
OUTPUT_DIR = "generated_thy_variants"
CIPHERS_NAME = ["Simon", "Speck"]

def generate_files(template_path, output_dir, setups, cipher_name):
    """Generate theory files for all variants of a cipher."""
    os.makedirs(output_dir, exist_ok=True)

    # Read template
    with open(template_path, "r") as f:
        template = f.read()

    # Generate files for all variants
    for block_size, key_size in setups:
        word_size = block_size // 2  # Compute word_size here
        
        # Replace placeholders
        content = template.replace("{block_size}", str(block_size)) \
                         .replace("{key_size}", str(key_size)) \
                         .replace("{word_size}", str(word_size))
        
        # Output file name
        output_file = os.path.join(output_dir, f"{cipher_name}_{block_size}_{key_size}.thy")

        # Write file
        with open(output_file, "w") as f:
            f.write(content)

        print(f" OK -- Generated {output_file}")

# Read template
for i in range(2):
    path = TEMPLATE_PATH[i]
    cipher_name = CIPHERS_NAME[i]
    # Generate Simon & Speck files
    generate_files(path, OUTPUT_DIR, simon_speck_setups, cipher_name)


 OK -- Generated generated_thy_variants/Simon_32_64.thy
 OK -- Generated generated_thy_variants/Simon_48_72.thy
 OK -- Generated generated_thy_variants/Simon_48_96.thy
 OK -- Generated generated_thy_variants/Simon_64_96.thy
 OK -- Generated generated_thy_variants/Simon_64_128.thy
 OK -- Generated generated_thy_variants/Simon_96_96.thy
 OK -- Generated generated_thy_variants/Simon_96_144.thy
 OK -- Generated generated_thy_variants/Simon_128_128.thy
 OK -- Generated generated_thy_variants/Simon_128_192.thy
 OK -- Generated generated_thy_variants/Simon_128_256.thy
 OK -- Generated generated_thy_variants/Speck_32_64.thy
 OK -- Generated generated_thy_variants/Speck_48_72.thy
 OK -- Generated generated_thy_variants/Speck_48_96.thy
 OK -- Generated generated_thy_variants/Speck_64_96.thy
 OK -- Generated generated_thy_variants/Speck_64_128.thy
 OK -- Generated generated_thy_variants/Speck_96_96.thy
 OK -- Generated generated_thy_variants/Speck_96_144.thy
 OK -- Generated generated_thy_variant

In [2]:
import os

# PRESENT variants with all required parameters
present_setups = [
    # (block_size, key_size, drop_bit, sbox_upper_pos, sbox_lower_pos, xor_position)
    (64, 80, 16, 76, None, 15),   # PRESENT-80: one S-box at bit 76, XOR at 15
    (64, 128, 64, 124, 120, 62),  # PRESENT-128: two S-boxes at 124 and 120, XOR at 62
]

TEMPLATE_PATH = "present_template.thy"
OUTPUT_DIR = "generated_present"

def generate_present_files(template_path, output_dir, setups):
    """Generate PRESENT theory files for all variants."""
    os.makedirs(output_dir, exist_ok=True)

    # Read template
    with open(template_path, "r") as f:
        template = f.read()

    # Generate files for all variants
    for block_size, key_size, drop_bit, sbox_upper, sbox_lower, xor_pos in setups:
        # Calculate derived values
        block_size_div_4 = block_size // 4
        block_size_minus_1 = block_size - 1
        
        # Build S-box application logic
        if sbox_lower is None:
            # PRESENT-80: single S-box
            sbox_application = f"""sbox_val_raw = word_slice {sbox_upper} 4 k_rot;
    sbox_val = present_sbox sbox_val_raw;"""
            clear_masks = f"""clear_mask1 = not (push_bit {sbox_upper} (mask 4));
    clear_mask2 = not (push_bit {xor_pos} (mask 5));
    clear_mask = and clear_mask1 clear_mask2;"""
            insert_operations = f"""k_sboxed_val = or k_cleared (push_bit {sbox_upper} (ucast sbox_val :: present_key));
    k_xored_val = or k_sboxed_val (push_bit {xor_pos} (ucast xor_val_update :: present_key))"""
        else:
            # PRESENT-128: two S-boxes
            sbox_application = f"""sbox_val1_raw = word_slice {sbox_upper} 4 k_rot;
    sbox_val1 = present_sbox sbox_val1_raw;
    sbox_val2_raw = word_slice {sbox_lower} 4 k_rot;
    sbox_val2 = present_sbox sbox_val2_raw;"""
            clear_masks = f"""clear_mask1 = not (push_bit {sbox_upper} (mask 4));
    clear_mask2 = not (push_bit {sbox_lower} (mask 4));
    clear_mask3 = not (push_bit {xor_pos} (mask 5));
    clear_mask = and clear_mask1 (and clear_mask2 clear_mask3);"""
            insert_operations = f"""k_sboxed1 = or k_cleared (push_bit {sbox_upper} (ucast sbox_val1 :: present_key));
    k_sboxed2 = or k_sboxed1 (push_bit {sbox_lower} (ucast sbox_val2 :: present_key));
    k_xored_val = or k_sboxed2 (push_bit {xor_pos} (ucast xor_val_update :: present_key))"""

        # Replace placeholders
        content = template.replace("{block_size}", str(block_size)) \
                         .replace("{key_size}", str(key_size)) \
                         .replace("{drop_bit}", str(drop_bit)) \
                         .replace("{block_size_div_4}", str(block_size_div_4)) \
                         .replace("{block_size_minus_1}", str(block_size_minus_1)) \
                         .replace("{sbox_application}", sbox_application) \
                         .replace("{clear_masks}", clear_masks) \
                         .replace("{insert_operations}", insert_operations) \
                         .replace("{xor_position}", str(xor_pos))
        
        # Output file name
        output_file = os.path.join(output_dir, f"Present_{block_size}_{key_size}.thy")

        # Write file
        with open(output_file, "w") as f:
            f.write(content)

        print(f" OK -- Generated {output_file}")

# Generate PRESENT files
generate_present_files(TEMPLATE_PATH, OUTPUT_DIR, present_setups)

 OK -- Generated generated_present/Present_64_80.thy
 OK -- Generated generated_present/Present_64_128.thy
